![20220719094116](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220719094116.png)

k-fold CV:

![20220719094414](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220719094414.png)

In [13]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')

lb = load_boston()
X = pd.DataFrame(lb.data,columns=lb.feature_names)
y = lb.target

X_train , X_test, y_train ,y_test = train_test_split(X,y,random_state=2022,test_size=0.2)


In [17]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
scores = cross_val_score(lr,X,y,scoring='r2',cv=5)
scores

array([ 0.63919994,  0.71386698,  0.58702344,  0.07923081, -0.25294154])

## CV迭代器

![20220719102631](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220719102631.png)

In [62]:
# K fold

import numpy as np
from sklearn.model_selection import KFold

X = np.arange(10)
kf = KFold(n_splits=10)
for train, test in kf.split(X):
    print(f"{train} {test}")

[1 2 3 4 5 6 7 8 9] [0]
[0 2 3 4 5 6 7 8 9] [1]
[0 1 3 4 5 6 7 8 9] [2]
[0 1 2 4 5 6 7 8 9] [3]
[0 1 2 3 5 6 7 8 9] [4]
[0 1 2 3 4 6 7 8 9] [5]
[0 1 2 3 4 5 7 8 9] [6]
[0 1 2 3 4 5 6 8 9] [7]
[0 1 2 3 4 5 6 7 9] [8]
[0 1 2 3 4 5 6 7 8] [9]


In [39]:
# Repeated K-Fold 重复K-Foldn次

import numpy as np
from sklearn.model_selection import RepeatedKFold

X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])

# 2次 k-fold
rkf = RepeatedKFold(n_splits=4, n_repeats=2, random_state=2022)
for train, test in rkf.split(X):
    print(f"{train} {test}")

[0 1 3] [2]
[0 1 2] [3]
[1 2 3] [0]
[0 2 3] [1]
[0 1 2] [3]
[0 1 3] [2]
[0 2 3] [1]
[1 2 3] [0]


![20220719102650](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220719102650.png)

ShuffleSplit 验证集可能会有重复

In [49]:
# 先shuffle数据集，再划分

from sklearn.model_selection import ShuffleSplit

X = np.arange(10)
ss = ShuffleSplit(n_splits=10, random_state=0)

for train_index, test_index in ss.split(X):
    print(f"{train_index} {test_index}")

[8 4 9 1 6 7 3 0 5] [2]
[5 1 2 9 8 0 6 7 4] [3]
[3 8 4 5 1 0 6 9 7] [2]
[1 9 2 7 5 8 0 3 4] [6]
[2 7 4 1 0 6 8 9 3] [5]
[8 9 5 2 6 3 7 4 0] [1]
[2 1 7 8 6 5 4 9 0] [3]
[9 2 3 5 6 7 1 8 4] [0]
[8 9 1 4 5 3 6 0 2] [7]
[2 5 0 8 3 7 9 6 4] [1]


In [55]:
# 基于类标签的分层划分
# 当负面样本可能比正面样本多几倍。
# 建议使用StratifiedKFold和StratifiedShuffleSplit中实现的分层抽样，以确保在每个训练和验证集中大致保持相对的类别频率

from sklearn.model_selection import StratifiedKFold, KFold
import numpy as np

X, y = np.ones((50, 1)), np.hstack(([0] * 45, [1] * 5))

print('-----分层kfold----------------')
skf = StratifiedKFold(n_splits=3)
for train, test in skf.split(X, y):
    print(f'train -  {np.bincount(y[train])}   |   test -  {np.bincount(y[test])}')


print('-----kfold--------------------')
kf = KFold(n_splits=3)
for train, test in kf.split(X, y):
    print(f'train -  {np.bincount(y[train])}   |   test -  {np.bincount(y[test])}')

-----分层kfold----------------
train -  [30  3]   |   test -  [15  2]
train -  [30  3]   |   test -  [15  2]
train -  [30  4]   |   test -  [15  1]
-----kfold--------------------
train -  [28  5]   |   test -  [17]
train -  [28  5]   |   test -  [17]
train -  [34]   |   test -  [11  5]


![20220719103900](https://cdn.jsdelivr.net/gh/xihuishawpy/PicBad@main/blogs/pictures/20220719103900.png)

In [60]:
# 基于时间序列的划分

from sklearn.model_selection import TimeSeriesSplit

X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([1, 2, 3, 4, 5, 6])
tscv = TimeSeriesSplit(n_splits=3)
print(tscv)

for train, test in tscv.split(X):
    print(f"{train} {test}")

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None)
[0 1 2] [3]
[0 1 2 3] [4]
[0 1 2 3 4] [5]
